# Frame the Problem

# Story
The United Kingdom government wants to devote a large portion of funding towards suprressing vehicular accidents. Before they go through with this proposed funding, they want to identify trends in the accident reports to see what areas of housing should the funding be rationalized out to. 


1. Define the objective in business terms. 
Utilizing the United Kingdom vehicle related accidents dataset, we want to to determine the causes of the least and most deaths each year.
2. How will your solution be used? 
Hopefully, by figuring out time frames of heightened vehicle accidents there can be extended efforts to help prevent such high numbers.
3. What are the current solutions/workarounds (if any)? 
None
4. How should you frame this problem (supervised/unsupervised, online/offline, ...)? 
This will be a supervised, classification offline learning problem.
5. How should performance be measured? Is the performance measure aligned with the business objective?
Because we want to know what timeframes contains the 

Recall
Precision
Accuracy
F1 Scoring

6. What would be the minimum performance needed to reach the business objective? 

7. What are comparable problems? Can you reuse experience or tools?

8. Is human expertise available?

9. How would you solve the problem manually?

10.  List the assumptions you (or others) have made so far. Verify assumptions if possible.
